In [18]:
import numpy as np
import matplotlib.pyplot as plt


class ErrorMetricsCalculator:
    def __init__(self, odom_data, filter_data, ground_truth_data):
        self.odom_data = odom_data
        self.filter_data = filter_data
        self.ground_truth_data = ground_truth_data
        
        self.odom_metrics_results = None
        self.filter_metrics_results = None

    def odom_metrics(self):
        odom = self.odom_data
        ground = self.ground_truth_data

        odom_xy = odom[:,:2]
        odom_yaw = odom[:,-1]

        ground_xy = ground[:,:2]
        ground_yaw = ground[:,-1]

        N = len(odom_xy)

        e_xy = np.linalg.norm(odom_xy-ground_xy, axis=-1)
        se_xy = np.square(e_xy)
        mse_xy = np.sum(se_xy)/N
        rmse_xy = np.sqrt(mse_xy)

        e_yaw = odom_yaw-ground_yaw
        se_yaw = np.square(e_yaw)
        mse_yaw = np.sum(se_yaw)/N
        rmse_yaw = np.sqrt(mse_yaw)

        RMSE_odom = [rmse_xy, rmse_yaw]
        MAE_odom = [np.max(e_xy), np.max(np.abs(e_yaw))]
        TCE_odom = [np.sum(e_xy), np.sum(np.abs(e_yaw))]

        odom_x = odom_xy[:,0]
        odom_y = odom_xy[:,-1]

        # Initialize total distance
        TDT_odom = 0.0

    # Iterate through points and calculate distances
        for i in range(0,N-1):
            
            delta_x = odom_x[i+1] - odom_x[i]
            delta_y = odom_y[i+1] - odom_y[i]
            distance = np.sqrt(delta_x**2 + delta_y**2)
            TDT_odom += distance
        
        e_p_odom = TCE_odom/TDT_odom
        
        self.odom_metrics_results = [RMSE_odom, MAE_odom, TCE_odom, e_p_odom[0]]
        return self.odom_metrics_results
        #return self.odom_metrics_results
        
    def filter_metrics(self, filter, ground):
        filter = self.filter_data
        ground = self.ground_truth_data

        filter_xy = filter[:,:2]
        filter_yaw = filter[:,-1]

        ground_xy = ground[:,:2]
        ground_yaw = ground[:,-1]

        N = len(filter_xy)

        e_xy = np.linalg.norm(filter_xy-ground_xy, axis=-1)
        se_xy = np.square(e_xy)
        mse_xy = np.sum(se_xy)/N
        rmse_xy = np.sqrt(mse_xy)

        e_yaw = filter_yaw-ground_yaw
        se_yaw = np.square(e_yaw)
        mse_yaw = np.sum(se_yaw)/N
        rmse_yaw = np.sqrt(mse_yaw)

        RMSE_filter = [rmse_xy, rmse_yaw]   
        MAE_filter = [np.max(e_xy), np.max(np.abs(e_yaw))]
        TCE_filter = [np.sum(e_xy), np.sum(np.abs(e_yaw))]

        filter_x = filter_xy[:,0]
        filter_y = filter_xy[:,-1]
        # Initialize total distance
        TDT_filter = 0.0

    # Iterate through points and calculate distances
        for i in range(0,N-1):
            
            delta_x = filter_x[i+1] - filter_x[i]
            delta_y = filter_y[i+1] - filter_y[i]
            distance = np.sqrt(delta_x**2 + delta_y**2)
            TDT_filter += distance

        print("Total Distance Traveled filter:", TDT_filter)
        
        e_p_filter = TCE_filter/TDT_filter

        self.filter_metrics_results = [RMSE_filter, MAE_filter, TCE_filter, e_p_filter]

        #return RMSE_filter, MAE_filter, TCE_filter, e_p_filter
    
    
    
    def plot_data(data, title):
        plt.figure(figsize=(10, 5))
        time = np.arange(len(data))

        plt.subplot(3, 1, 1)
        plt.plot(time, data[:, 0], label='X')
        plt.title(f'{title} - X')
        plt.xlabel('Time')
        plt.ylabel('X')

        plt.subplot(3, 1, 2)
        plt.plot(time, data[:, 1], label='Y')
        plt.title(f'{title} - Y')
        plt.xlabel('Time')
        plt.ylabel('Y')

        plt.subplot(3, 1, 3)
        plt.plot(time, data[:, 2], label='Yaw')
        plt.title(f'{title} - Yaw')
        plt.xlabel('Time')
        plt.ylabel('Yaw')

        plt.tight_layout()
        plt.show()
        #print metrics

        #odom_metrics = self.odom_metrics_results
        #filter_metrics = filter_metrics(odom_data, ground_truth_data)

    """
    def compare_results(self):

        odom_metrics_results = self.odom_metrics_results
        filter_metrics_results = self.filter_metrics_results
        RMSE_diff = odom_metrics_results[0]-filter_metrics_results[0]
        MAE_diff = odom_metrics_results[1]-filter_metrics_results[1]
        TCE_diff = odom_metrics_results[2]-filter_metrics_results[2]
        ep_diff = odom_metrics_results[3]-filter_metrics_results[3]
        print("RMSE_diff", RMSE_diff)
        return RMSE_diff, MAE_diff, TCE_diff, ep_diff
    """


    """

    print("Loaded Odom Data:")
    print(odom_data)
    print("odom data shape", odom_data.shape)

    print("Loaded Filter Data:")
    print(filter_data)
    print("filter data shape", filter_data.shape)


    print("Loaded Ground Truth Data:")
    print(ground_truth_data)
    print("ground_truth data shape", ground_truth_data.shape)


    print(f"\n\n%%%%% ERROR METRICS FOR /diff_drive_controller/odom AND /odometry/filtererd %%%%%\n"
        f"odom_metric:\n>RMSE_odom = {odom_metrics[1]}\n>MAE_odom = {odom_metrics[2]}\n>Total cumulative error odom (TCE_odom) = {odom_metrics[3]}\n%>err final pos w.r.t. the total distance = {odom_metrics[4]}\n"
        f"filter_metric:\n>RMSE_filter = {filter_metrics[1]}\n>MAE_filter = {filter_metrics[2]}\nT>otal cumulative error filter (TCE_filter) = {filter_metrics[3]}\n%>err final pos w.r.t. the total distance = {filter_metrics[4]}\n")
        #comparison

    print("odom_metrics:", odom_metrics)
    print("filter_metrics:", filter_metrics)


    print("try difference")
    print(odom_data-ground_truth_data)
    """

In [21]:
#if __name__ == "__main__":
odom_data = np.load("odom.npy")
filter_data = np.load("filter.npy")
ground_truth_data = np.load("ground_truth.npy")
calculator = ErrorMetricsCalculator(odom_data, filter_data, ground_truth_data)
calculator.odom_metrics()
calculator.odom_metrics_results

[[0.1635337715199515, 0.032603534501449],
 [0.31745994775823055, 0.061345246536836484],
 [42.89837953154924, 9.198637668124682],
 1.2927592011870468]